In [22]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\model\\modelAI.py'>

In [23]:
## parameters
indicatorCodeAI = 'CBI2_01'
month = 'JULIO'
month_number = '2022-07'
year = 2022
test = False
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------CBI2_01------------------------------------------------------


In [24]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: csdtthikyov11so4
parentFormIdAI: cl67fi2kyov11sn3
indicatorIdAI: cbuoigikylzqu4h3y
indicatorIdsOsmosys: [90 83]


In [25]:

print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))


indicatorIdsOsmosys: [90 83]
reported data: 12


In [26]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [27]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]

    CBI2_01_RM_NA_PR = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    CBI2_01_RM_NN_PR = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    CBI2_01_RM_MM_PR = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    CBI2_01_RM_HH_PR = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    CBI2_01_CA_NA_PR = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    CBI2_01_CA_NN_PR = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    CBI2_01_CA_MM_PR = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    CBI2_01_CA_HH_PR = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[
        0].value_a

    usd_transfer = int(osmosys.osmosys.getCBIBudget(year=year, month=month, orgOsmosys=orgAcron,
                                                    indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                    cantonCode=cantonCode).iloc[0].budget)

    subform = model.modelAI.SubFormACBI2_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        espacion_apoyo='No',
        poblacion_meta=poblacion_meta,
        poblacion_meta_freq='Primera vez',
        modalidad_impl='c3jf6gykykwwemuk7',  ## EFECTIVO
        mecanismos='c3de3pmkym0dbino',  ## cajero
        transferencia='Incondicional',
        usd_transfer=usd_transfer,
        CBI2_01_RM_NA_PR=CBI2_01_RM_NA_PR,
        CBI2_01_RM_NN_PR=CBI2_01_RM_NN_PR,
        CBI2_01_RM_MM_PR=CBI2_01_RM_MM_PR,
        CBI2_01_RM_HH_PR=CBI2_01_RM_HH_PR,
        CBI2_01_CA_NA_PR=CBI2_01_CA_NA_PR,
        CBI2_01_CA_NN_PR=CBI2_01_CA_NN_PR,
        CBI2_01_CA_MM_PR=CBI2_01_CA_MM_PR,
        CBI2_01_CA_HH_PR=CBI2_01_CA_HH_PR,
        CBI2_01_CUAL = commentary

    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))


    SELECT 
    v.ie_id	, ie.performance_indicator_id indicator_osmosys_id,
     v.month_id	,v.year	,v.month_year_order	,v.month	,
    org.id org_odmodyd_id,org.acronym osmosys_org, v.canton_code	,
    v.canton	,v.population_type	,v.age_gender	,
    sum(v.value) value_a
    FROM ai_integration.ai_adultos_ninos_ref_migrantes_ven v
    INNER JOIN osmosys.indicator_executions ie on v.ie_id=ie.id
    INNER JOIN osmosys.projects pr on ie.project_id=pr.id
    INNER JOIN osmosys.organizations org on pr.organization_id=org."id"
    WHERE v.year=2022 and v.month='JULIO' 
    and org.acronym='HIAS'
    and v.canton_code='0805'
    and ie.performance_indicator_id in (90,83)
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
    

    SELECT 
    v.ie_id	, ie.performance_indicator_id indicator_osmosys_id,
     v.month_id	,v.year	,v.month_year_order	,v.month	,
    org.id org_odmodyd_id,org.acronym osmosys_org, v.canton_code	,
    v.canton	,v.population_type	,v.age_gender	,
    sum(v.value) value_a
    FROM a

In [20]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

In [21]:
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + '-' + str(indicatorIdsOsmosys) + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: JULIO\data_CBI2_01-[90 83].json


In [9]:
## envio a AI
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '-' + str(
            indicatorIdsOsmosys) + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '-' + str(
                    indicatorIdsOsmosys) + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

--------------------------------------CBI2_01-[90]------------------------------------------------------
